In [1]:
import sys
from elasticsearch import Elasticsearch
import json
import time
import sys
# from elasticsearch import Elasticsearch
from elasticsearch.helpers import bulk
import csv
from tqdm import tqdm
import pandas as pd
# import tensorflow as tf
# import tensorflow_hub as hub

In [2]:
# df=pd.read_json("/Users/amansawarn/Downloads/questions_w_embeddings (1).json")
# df=df.to_dict('records')

In [7]:
questions=pd.read_csv(r"/Users/amansawarn/Downloads/Questions.csv", encoding='ISO-8859-1')
tags=pd.read_csv(r"/Users/amansawarn/Downloads/Tags.csv", encoding='ISO-8859-1')
answers=pd.read_csv(r"/Users/amansawarn/Downloads/Answers.csv", encoding='ISO-8859-1')


In [8]:
questions.shape, tags.shape, answers.shape

((1264216, 7), (3750994, 2), (2014516, 6))

In [9]:
tags['Tag'] = tags['Tag'].astype(str)
grouped_tags = tags.groupby("Id")['Tag'].apply(lambda tags: ' '.join(tags))
grouped_tags.reset_index()
tags_final = pd.DataFrame({'Id':grouped_tags.index, 'Tags':grouped_tags.values})
questions_with_tags = questions.merge(tags_final, on='Id')
questions_with_tags

,Id,OwnerUserId,CreationDate,ClosedDate,Score,Title,Body,Tags
0,80,26.0,2008-08-01T13:57:07Z,NaN,26,SQLStatement.execute() - multiple queries in o...,<p>I've written a database generation script i...,flex actionscript-3 air
1,90,58.0,2008-08-01T14:41:24Z,2012-12-26T03:45:49Z,144,Good branching and merging tutorials for Torto...,<p>Are there any really good tutorials explain...,svn tortoisesvn branch branching-and-merging
2,120,83.0,2008-08-01T15:50:08Z,NaN,21,ASP.NET Site Maps,<p>Has anyone got experience creating <strong>...,sql asp.net sitemap
3,180,2089740.0,2008-08-01T18:42:19Z,NaN,53,Function for creating color wheels,<p>This is something I've pseudo-solved many t...,algorithm language-agnostic colors color-space
4,260,91.0,2008-08-01T23:22:08Z,NaN,49,Adding scripting functionality to .NET applica...,<p>I have a little game written in C#. It uses...,c# .net scripting compiler-construction
...,...,...,...,...,...,...,...,...
1264211,40143210,5610777.0,2016-10-19T23:38:01Z,NaN,0,URL routing in PHP (MVC),<p>I am building a custom MVC project and I ha...,php .htaccess
1264212,40143300,3791161.0,2016-10-19T23:48:09Z,NaN,0,Bigquery.Jobs.Insert - Resumable Upload?,<p>The API docs show that you should be able t...,google-bigquery
1264213,40143340,7028647.0,2016-10-19T23:52:50Z,NaN,1,Obfuscating code in android studio,<p>Under minifyEnabled I changed from false to...,android android-studio
1264214,40143360,871677.0,2016-10-19T23:55:24Z,NaN,0,How to fire function after v-model change?,<p>I have input which I use to filter my array...,javascript vue.js


In [10]:
questions.shape, tags.shape, answers.shape, questions_with_tags.shape

((1264216, 7), (3750994, 2), (2014516, 6), (1264216, 8))

In [11]:
questions_with_tags['title_body']=questions_with_tags['Title'] + " "+questions_with_tags['Body']

In [12]:
df = questions_with_tags[['Id', 'Title',
       'Body', 'title_body']]

In [13]:
df.shape

(1264216, 4)

In [14]:
df=df.sample(30000)

In [19]:
questions_with_tags.head()

,Id,OwnerUserId,CreationDate,ClosedDate,Score,Title,Body,Tags,title_body
0,80,26.0,2008-08-01T13:57:07Z,NaN,26,SQLStatement.execute() - multiple queries in o...,<p>I've written a database generation script i...,flex actionscript-3 air,SQLStatement.execute() - multiple queries in o...
1,90,58.0,2008-08-01T14:41:24Z,2012-12-26T03:45:49Z,144,Good branching and merging tutorials for Torto...,<p>Are there any really good tutorials explain...,svn tortoisesvn branch branching-and-merging,Good branching and merging tutorials for Torto...
2,120,83.0,2008-08-01T15:50:08Z,NaN,21,ASP.NET Site Maps,<p>Has anyone got experience creating <strong>...,sql asp.net sitemap,ASP.NET Site Maps <p>Has anyone got experience...
3,180,2089740.0,2008-08-01T18:42:19Z,NaN,53,Function for creating color wheels,<p>This is something I've pseudo-solved many t...,algorithm language-agnostic colors color-space,Function for creating color wheels <p>This is ...
4,260,91.0,2008-08-01T23:22:08Z,NaN,49,Adding scripting functionality to .NET applica...,<p>I have a little game written in C#. It uses...,c# .net scripting compiler-construction,Adding scripting functionality to .NET applica...


In [20]:
questions_with_tags['OwnerUserId'].value_counts()

875317.0     250
39677.0      227
4653.0       175
651174.0     164
34537.0      153
            ... 
2817145.0      1
3000627.0      1
3000470.0      1
1521659.0      1
6823982.0      1
Name: OwnerUserId, Length: 630909, dtype: int64

In [21]:
questions_with_tags[questions_with_tags['OwnerUserId']==875317.0]

,Id,OwnerUserId,CreationDate,ClosedDate,Score,Title,Body,Tags,title_body
209813,7948030,875317.0,2011-10-30T21:39:13Z,2013-03-03T14:42:19Z,0,"Shouldn't the Manifest file be named ""AndroidM...",<p>I'm getting a nonsensical error message:</p...,android android-manifest,"Shouldn't the Manifest file be named ""AndroidM..."
214102,8088820,875317.0,2011-11-11T01:57:17Z,NaN,17,"Can I set Eclipse to only emit errors, warning...",<p>I want to be able to ignore warnings in oth...,eclipse customization,"Can I set Eclipse to only emit errors, warning..."
217361,8199310,875317.0,2011-11-20T03:47:43Z,NaN,0,The Case of the Incredible Disappearing Button,<p>I have a button that I declare in main.xml;...,android android-layout android-emulator androi...,The Case of the Incredible Disappearing Button...
219742,8277170,875317.0,2011-11-26T08:12:10Z,NaN,1,How can I run my app without debugging?,<p>I'm running two Android apps in different e...,android android-emulator eclipse-plugin,How can I run my app without debugging? <p>I'm...
224220,8425690,875317.0,2011-12-08T03:16:35Z,NaN,0,Why is my value saved to SharedPreferences in ...,<p>I have the following code:</p>\n\n<pre><cod...,android android-emulator android-widget androi...,Why is my value saved to SharedPreferences in ...
...,...,...,...,...,...,...,...,...,...
1243282,39645440,875317.0,2016-09-22T17:35:09Z,NaN,0,How can I cause a legend to appear to the righ...,<p>I'm creating a fairly simple pie chart with...,chart.js pie-chart chart.js2,How can I cause a legend to appear to the righ...
1243321,39646320,875317.0,2016-09-22T18:25:17Z,NaN,0,How can I show a subset of data on pie pieces ...,<p>I've got a pie chart that looks like this w...,javascript drawing chart.js pie-chart chart.js2,How can I show a subset of data on pie pieces ...
1248697,39779510,875317.0,2016-09-29T20:34:01Z,NaN,0,How can I calculate a value and place it insid...,<p>Since it seems impossible or highly difficu...,javascript jquery-plugins bar-chart chart.js c...,How can I calculate a value and place it insid...
1254885,39925460,875317.0,2016-10-07T20:39:21Z,NaN,12,How can I filter columns (rather than rows) in...,<p>Filtering rows in a particular column are e...,c# excel excel-interop epplus epplus-4,How can I filter columns (rather than rows) in...


In [25]:
one_user_questions=questions_with_tags[questions_with_tags['OwnerUserId']==875317.0]['Id'].to_list()
print(one_user_questions)

[7948030, 8088820, 8199310, 8277170, 8425690, 8460990, 8556920, 8640530, 8650600, 8675080, 8737190, 8737270, 8778850, 8949900, 9122560, 9135110, 9153650, 9264090, 9284610, 9302050, 9753960, 9830940, 9897790, 9934900, 10220010, 10324780, 10377130, 10438220, 10689140, 10901590, 10902620, 10952470, 10990520, 11006100, 11386670, 11508240, 11514640, 11614800, 11857700, 11860540, 11939460, 11995470, 11996060, 12046810, 12201620, 12270710, 12410630, 12610160, 12644050, 13036850, 13130930, 13180180, 13517590, 13661440, 13769240, 13851460, 14007390, 14104040, 14342390, 14945720, 15077220, 15312500, 15313230, 15330310, 15503570, 15774160, 15912410, 15980680, 16019210, 16158440, 16406940, 16484560, 16509150, 16513530, 16547960, 16843420, 16991020, 17112660, 17227920, 17371790, 17391430, 17414390, 17431660, 17432960, 17540480, 17581330, 17599630, 17631690, 17758320, 17776800, 17952900, 17955040, 18242690, 18324370, 18346600, 18601400, 18857350, 18967210, 18991450, 19011120, 19525590, 19552970, 198

In [84]:
# df.to_json("questions_titles90k.json", orient='records', indent=4)